## Tool calling

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. 

* NOTE : Despite the name, the does not execute the action! The model comes up with the arguments to a tool (of the provided tooks), then actually running the tool is up to the user (in this case developers).

In [1]:
from langchain_core.tools import tool

Tool creation with funtions

In [2]:
@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location.
    Args:
        location: The location to get the weather for.
    """
    return f"The current weather in {location} is sunny."

@tool
def get_weather_forecast(location: str) -> str:
    """Get the weather forecast for a given location.
    Args:
        location: The location to get the weather forecast for.
    """
    return f"The weather forecast for {location} is sunny and hot."

@tool
def get_holidays(location: str) -> str:
    """Get the upcoming holidays for a given location.
    Args:
        location: The location to get the upcoming holidays for.
    """
    return f"The upcoming holidays in {location} are holiday1, holiday2, and holiday3."

@tool
def get_currency_exchange(from_currency: str, to_currency: str) -> str:
    """Get the exchange rate between two currencies.
    Args:
        from_currency: The currency to convert from.
        to_currency: The currency to convert to.
    """
    return f"The exchange rate between {from_currency} and {to_currency} is 1.0."


In [3]:
tools  = [get_current_weather, get_weather_forecast, get_holidays, get_currency_exchange]

In [4]:
tool_look_up = {
    "get_current_weather": get_current_weather, 
    "get_weather_forecast": get_weather_forecast, 
    "get_holidays": get_holidays, 
    "get_currency_exchange": get_currency_exchange
    }

Defining tool schemas with Pydantic class

In [62]:
# from pydantic import BaseModel, Field

# class get_current_weather(BaseModel):
#     """Get the current weather in a given location."""
#     location: str = Field(..., description="The location to get the weather for.")
    
# class get_weather_forecast(BaseModel):
#     """Get the weather forecast for a given location."""
#     location: str = Field(..., description="The location to get the weather forecast for.")
    
# class get_holidays(BaseModel):
#     """Get the upcoming holidays for a given location."""
#     location: str = Field(..., description="The location to get the upcoming holidays for.")
    
# class get_currency_exchange(BaseModel):
#     """Get the exchange rate between two currencies."""
#     from_currency: str = Field(..., description="The currency to convert from.")
#     to_currency: str = Field(..., description="The currency to convert to.")

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
query = "I am planning to move to Cananda ,from Kenya, for my masters degree. what is the wather in Canada,and exchange rate between my currency and thiers?"

In [9]:
chain = llm_with_tools 
reponse = chain.invoke(query)

In [10]:
reponse

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HFCAVLq8C7yxqcC4EojjEDyl', 'function': {'arguments': '{"location": "Canada"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_mrQf9acYwNVgmLX2TawWntuA', 'function': {'arguments': '{"from_currency": "KES", "to_currency": "CAD"}', 'name': 'get_currency_exchange'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 230, 'total_tokens': 282, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4421ae8d-2d70-4249-afbc-452d769da723-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Canada'}, 'id': 'call_HFCAVLq8C7yxqcC4EojjEDyl', 'type': 'tool_call'}, {'name': 'get_currency_exchange', 'args': {'from_currency': 

In [12]:
reponse.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': 'Canada'},
  'id': 'call_HFCAVLq8C7yxqcC4EojjEDyl',
  'type': 'tool_call'},
 {'name': 'get_currency_exchange',
  'args': {'from_currency': 'KES', 'to_currency': 'CAD'},
  'id': 'call_mrQf9acYwNVgmLX2TawWntuA',
  'type': 'tool_call'}]

In [13]:
for tool_call in reponse.tool_calls:
    selected_tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool = tool_look_up[selected_tool_name]
    tool_response = tool(tool_args)
    
    print(f"Tool: {tool}")
    print(f"Arguments: {tool_args}")
    print()
    print(f"Response: {tool_response}")
    print()
    print()

Tool: name='get_current_weather' description='Get the current weather in a given location.\n    Args:\n        location: The location to get the weather for.' args_schema=<class 'langchain_core.utils.pydantic.get_current_weather'> func=<function get_current_weather at 0x7fdeac20fe20>
Arguments: {'location': 'Canada'}

Response: The current weather in Canada is sunny.


Tool: name='get_currency_exchange' description='Get the exchange rate between two currencies.\n    Args:\n        from_currency: The currency to convert from.\n        to_currency: The currency to convert to.' args_schema=<class 'langchain_core.utils.pydantic.get_currency_exchange'> func=<function get_currency_exchange at 0x7fdebc146840>
Arguments: {'from_currency': 'KES', 'to_currency': 'CAD'}

Response: The exchange rate between KES and CAD is 1.0.




/tmp/ipykernel_37716/948009541.py:5: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  tool_response = tool(tool_args)
